In [179]:
import os
file_path = os.path.join(os.getcwd() , 'kaggle.json')

In [180]:
# data handling
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

import os
from os.path import join

import pandas as pd
import numpy as np

from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
import lightgbm as lgb

import xgboost as xgb
import lightgbm as lgb

# prevent overfit
from sklearn.model_selection import train_test_split

# metrics
from sklearn.metrics import mean_squared_error

In [181]:
tr = pd.read_csv(r'C:\Users\dlfrn\learning\data\train.csv')
te = pd.read_csv(r'C:\Users\dlfrn\learning\data\test.csv')

In [182]:
# date preprocess
def date_preprocess(df):
    df['date'] = df['date'].apply(lambda i: i[:6]).astype(int)
    return df

def log_preprocess(df):
    df[['bedrooms', 'sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement']] = df[['bedrooms', 'sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement']].apply(lambda x: np.log1p(x))
    return df



In [183]:
tr = date_preprocess(tr)
te = date_preprocess(te)

tr = log_preprocess(tr)
te = log_preprocess(te)

y = np.log1p(tr['price'])
del tr['price']

In [184]:
te

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,15035,201412,1.386294,2.25,7.852050,8.887791,2.0,0,0,3,7,7.682943,5.993961,1951,1991,98125,47.7210,-122.319,1690,7639
1,15036,201412,1.609438,3.00,7.581210,8.517393,1.0,0,0,5,7,6.957497,6.814543,1965,0,98136,47.5208,-122.393,1360,5000
2,15037,201405,1.609438,4.50,8.598036,11.532051,1.0,0,0,3,11,8.266421,7.333676,2001,0,98053,47.6561,-122.005,4760,101930
3,15038,201504,1.386294,1.00,7.484930,8.918784,1.0,0,0,3,7,6.957497,6.594413,1960,0,98146,47.5123,-122.337,1780,8113
4,15039,201503,1.386294,2.50,7.544861,8.788898,2.0,0,0,3,7,7.544861,0.000000,2003,0,98038,47.3684,-122.031,2390,7570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6463,21498,201406,1.386294,1.75,7.313887,9.390075,1.0,0,0,3,6,7.313887,0.000000,2014,0,98010,47.3095,-122.002,1320,11303
6464,21499,201501,1.386294,2.00,7.307202,7.027315,3.0,0,0,3,8,7.307202,0.000000,2014,0,98144,47.5699,-122.288,1400,1230
6465,21500,201502,1.386294,2.50,7.178545,7.166266,2.0,0,0,3,8,7.074117,4.875197,2008,0,98116,47.5773,-122.409,1330,1265
6466,21501,201406,1.098612,0.75,6.928538,7.208600,2.0,0,0,3,7,6.928538,0.000000,2009,0,98144,47.5944,-122.299,1020,2007


In [185]:
del tr['id']

In [186]:
del te['id']

In [187]:
X_train, X_test, y_train, y_test = train_test_split(
    tr, y, test_size = 0.2, random_state=42
)

In [188]:
def rmse(y_test, y_pred):
    return np.sqrt(mean_squared_error(np.expm1(y_test),np.expm1(y_pred)))
def get_scores(models):
    df = {}
    for model in models:
        model_name = model.__class__.__name__
        model.fit(X_train, y)
        y_pred = model.predict(X_test)
        df[model_name] = rmse(y_test, y_pred)
        score_df = pd.DataFrame(df, index=['RMSE']).T.sort_values('RMSE', ascending=False)
        return score_df
    
def my_GridSearch(model, param_grid, verbose=2, n_jobs=5):
    grid_model = GridSearchCV(model, param_grid=param_grid, scoring='neg_mean_squared_error',
                              cv=5, verbose=verbose, n_jobs=n_jobs)
    grid_model.fit(tr, y)
    params = grid_model.cv_results_['params']
    score = grid_model.cv_results_['mean_test_score']
    
    results = pd.DataFrame(params)
    results['score'] = score
    
    results['RMSLE'] = np.sqrt(-1 * results['score'])
    results = results.sort_values('RMSLE')
    
    return results

def best_predict(model, grid_result):
    model = model(max_depth=grid_result['max_depth'][0],n_estimators=grid_result['n_estimators'][0], random_state=random_state)
    model.fit(tr, y)
    predictions = model.predict(te)
    predictions = np.expm1(predictions)
    return predictions

In [189]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

random_state = 42
gboost = GradientBoostingRegressor(random_state=random_state)
xgboost = xgb.XGBRegressor(random_state=random_state)
lightgbm = lgb.LGBMRegressor(random_state=random_state)
rdforest = RandomForestRegressor(random_state=random_state)

models = [gboost, xgboost, lightgbm, rdforest]

In [190]:
pred_df = pd.DataFrame()
predictions = np.zeros(len(te))

In [191]:
light_param = {
    "objective" : "regression",
    "metric" : "rmse",
    "boosting": 'gbdt',
    "max_depth" : -1,
    "num_leaves" : 5,
    "min_data_in_leaf": 5,
    "bagging_freq": 5,
    "learning_rate" : 0.01,
    "bagging_fraction" : 0.464,
    "feature_fraction" : 0.582,
    "tree_learner": "serial",
    "boost_from_average": "false",
    "verbosity" : 1,
    "feature_fraction_seed" : random_state,
    "bagging_fraction_seed" : random_state,
    "random_state": random_state
}  
lgb_train = lgb.Dataset(X_train, label=y_train)
lgb_valid = lgb.Dataset(X_test, label=y_test)

clf = lgb.train(light_param, lgb_train, 100000, valid_sets = [lgb_train, lgb_valid], early_stopping_rounds = 10000, verbose_eval=10000)
pred_df['LGBM'] = clf.predict(te, num_iteration=clf.best_iteration)
pred_df['LGBM'] = pred_df.apply(lambda x: np.expm1(x))
predictions += clf.predict(te, num_iteration=clf.best_iteration)
predictions = np.expm1(predictions)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000945 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2298
[LightGBM] [Info] Number of data points in the train set: 12028, number of used features: 19
Training until validation scores don't improve for 10000 rounds
[10000]	training's rmse: 0.133945	valid_1's rmse: 0.162744
[20000]	training's rmse: 0.117353	valid_1's rmse: 0.161764
Early stopping, best iteration is:
[18956]	training's rmse: 0.11877	valid_1's rmse: 0.161536


In [192]:
# GBM
param_grid = {
    'n_estimators': [50, 100, 200, 400],
    'max_depth': [1, 8, 16, 32]
}
grid_result = my_GridSearch(gboost, param_grid, verbose=2, n_jobs=5)
grid_result = grid_result.reset_index(drop=True)
predictions = best_predict(GradientBoostingRegressor, grid_result)
pred_df['GBR'] = predictions

Fitting 5 folds for each of 16 candidates, totalling 80 fits


In [193]:
# XGBR
param_grid = {
    'n_estimators': [50, 100, 200, 400],
    'max_depth': [1, 8, 16, 32]
}
grid_result = my_GridSearch(xgboost, param_grid, verbose=2, n_jobs=5)
grid_result = grid_result.reset_index(drop=True)
predictions = best_predict(XGBRegressor,grid_result)
pred_df['XGB'] = predictions

Fitting 5 folds for each of 16 candidates, totalling 80 fits


In [194]:
param_grid = {
    'n_estimators': [50, 100, 200, 400],
    'max_depth': [1, 8, 16, 32]
}
grid_result = my_GridSearch(rdforest, param_grid, verbose=2, n_jobs=5)
grid_result = grid_result.reset_index(drop=True)
predictions = best_predict(RandomForestRegressor,grid_result)
pred_df['RF'] = predictions

Fitting 5 folds for each of 16 candidates, totalling 80 fits


In [195]:
predictions = (pred_df.LGBM + pred_df.GBR + pred_df.XGB + pred_df.RF) / 4

In [196]:
predictions = pred_df.LGBM

In [197]:
submission = pd.read_csv(r'C:\Users\dlfrn\learning\submission.csv')
submission['price'] = predictions
submission.to_csv("submission.csv",index=False)
!kaggle competitions submit -c 2019-2nd-ml-month-with-kakr -f submission.csv -m "Message"

Successfully submitted to 2019 2nd ML month with KaKR



  0%|          | 0.00/160k [00:00<?, ?B/s]
  5%|▌         | 8.00k/160k [00:00<00:02, 64.6kB/s]
 55%|█████▌    | 88.0k/160k [00:02<00:02, 36.5kB/s]
 65%|██████▌   | 104k/160k [00:02<00:01, 42.4kB/s] 
 80%|████████  | 128k/160k [00:02<00:00, 55.2kB/s]
100%|██████████| 160k/160k [00:02<00:00, 77.9kB/s]
100%|██████████| 160k/160k [00:04<00:00, 36.4kB/s]


![image](.\[E-05]\result.png)